<a href="https://colab.research.google.com/github/pedromendesjr/estudos_ia/blob/main/Clonagem_de_Tom_de_Voz_Usando_IAs_Gratuitas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install groq --quiet
!pip install httpx==0.27.2 --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.1/109.1 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 1.8 MB/s eta 0:00:00


Pegue a API key aqui: https://console.groq.com/keys

In [ ]:
from groq import Groq
from google.colab import userdata
from time import sleep
GROQ_API_KEY = userdata.get('GROQ_API_KEY')

In [ ]:
def clonaTomDeVoz(artigos: list, groq_api_key: str, model_name: str):
  system_prompt = "You help users to mimic writing styles"
  if len(artigos) <= 6:
      trechosArtigos = "\n".join([f"<text>\n{artigo}\n</text>" for artigo in artigos[:1]])
      prompt1 = f"""
      Given the following sample text, please analyze and identify the author's distinctive stylistic elements in terms of tone, diction, sentence structure, burstiness, point of view, pace, literary devices and techniques, mood, perspective on time, personality, common enemies and text structure. Print them in a bullet list.
      {trechosArtigos}

      Return ONLY the stylistics.
      """

      client = Groq(api_key=GROQ_API_KEY)
      messages=[{"role": "system", "content": system_prompt}]

      # Pegar o estilo (tom de comunicação, dicçao etc.)
      print("(1/6) Aprendendo o estilo de escrita...")
      messages.append({"role": "user", "content": prompt1})
      completion = client.chat.completions.create(
                  model=model_name,
                  messages = messages
                  )
      stylistics = completion.choices[0].message.content
      messages.append({"role": "assistant", "content": stylistics})

      # Pegar exemplos de trechos e comentar o que está sendo utilizado
      sleep(15)
      print("(2/6) Comentando exemplos de utilização do estilo...")
      prompt2 = f"""
      Now i need you to identify 5 segment of the text provided that captures the unique writing style of this author. Each segment can be up to 400 words verbatim and span multiple paragraph. Choose a good chunk of the text so that it can be understood in context

      After each segment add a short commentary on why it's chosen. wrap each segment with a example tag, ie:

      <example>
      segment
      </example>

      commentary

      <example>
      segment
      </example>

      commentary


      Return ONLY the content following this <example>segment</example> commentary format. ONLY this.
      The commentary must put emphasis on the elements of stylistcs used in each excerpt.
      """
      messages.append({"role": "user", "content": prompt2})
      completion = client.chat.completions.create(
                  model=model_name,
                  messages = messages
                  )
      examples = completion.choices[0].message.content
      messages.append({"role": "assistant", "content": examples})
      sleep(15)
      sleep(15)
      # Comparando a saída final com o exemplo de teste
      print("(3/6) Gerando o título para o último artigo")
      ultimoArtigo = artigos[-1]
      prompt_comparacao = f"Generate a 6 words title for the following article: {ultimoArtigo}. Return ONLY the title, nothing more."
      messages.append({"role": "user", "content": prompt_comparacao})
      completion = client.chat.completions.create(
                  model=model_name,
                  messages = messages
                  )
      tituloUltimoArtigo = completion.choices[0].message.content
      messages.append({"role": "assistant", "content": tituloUltimoArtigo})
      prompt3 = f"""
      I need you to mimic the writing style of an author. Write with the following stylistic elements and use the examples provided to understand to apply the style.

      ## Stylistic Elements

      {stylistics}

      ## Examples

      {examples}
      """
      prompt3_completo = prompt3 + f"""

      # IMPORTANT
      Now, using this style, write a article titled: "{tituloUltimoArtigo}". Return ONLY the article. Maximum of 5500 tokens. No more."""
      messages.append({"role": "user", "content": prompt3_completo})
      print("(4/6) Criando o artigo utilizando o primeiro estilo de comunicação")
      completion = client.chat.completions.create(
                  model=model_name,
                  messages = messages
                  )
      articleExample = completion.choices[0].message.content
      messages.append({"role": "assistant", "content": articleExample})
      sleep(15)
      sleep(15)
      # Comparando e refinando
      print("(5/6) Refinando o prompt")
      prompt4 = f"""
      Compare that with the following text and highlight key differences in stylistic elements:

      {articleExample}
      """
      messages.append({"role": "user", "content": prompt4})
      completion = client.chat.completions.create(
                  model=model_name,
                  messages = messages
                  )
      completeToneCloned = completion.choices[0].message.content
      messages.append({"role": "assistant", "content": completeToneCloned})


      # Final result
      print("(6/6) Final result")
      prompt5 = f"""
      Now, create a new stylistcs adding the key differences, to make it more complete! Merge in a more condense text, using only keywords.

      Original stylistics:
      {stylistics}

      Remember: write the new complete stylistics in bulletpoints. Maximum of 6000 tokens. No more.
      """
      messages.append({"role": "user", "content": prompt5})
      completion = client.chat.completions.create(
                  model=model_name,
                  messages = messages
                  )

      finalAnswer = """
      I need you to mimic the writing style of an author. Write with the following stylistic elements and use the examples provided to understand to apply the style.

      ## Stylistic Elements

"""
      finalAnswer += completion.choices[0].message.content
      messages.append({"role": "assistant", "content": finalAnswer})

      finalAnswer += "\n\n ## Examples\n" + examples

      finalAnswer += """\n\n## YOUR JOB\nWrite an article about "[TOPICO]". Here is the context: "[CONTEXTO]". Wirte in pt-BR."""

      with open("toneOfVoice.txt", "w") as arquivo:
          arquivo.write(finalAnswer)

  else:
      print("No máximo, 6 artigos de exemplo.")

In [ ]:
artigos = [
    """
Vamos lá, é bem provável que nas últimas duas semanas o tema não tenha sido outro: o preço dos combustíveis. Ele foi o responsável pela greve dos caminhoneiros, pela quase-queda do governo e por uma quantidade enorme de medidas emergenciais.

Tem quem defenda os caminhoneiros, quem aponte que, ao final das contas, será o povo que arcará com as novas diretrizes de preços e quem quer que tudo isso se exploda.

O objetivo desse texto não é político, mas econômico. O que faz o preço dos combustíveis subir tanto?

Os elementos são tantos que é bem provável que eu esqueça de adicionar algumas informações. Paciência.

Bom, vamos lá: para explicar a dinâmica do preço dos combustíveis, temos que dividir o tema em sete verticais: Brasil, Estados Unidos, OPEP, normas e legislações, Irã, juros e câmbio.

A primeira pergunta que nos vêm à cabeça, é: por que diabos o combustível do mundo é negociado em dólar?

Porque, há mais de cinquenta anos, o Dólar é a moeda de reserva global. E por que? Porque ela venceu a livre concorrência.

Assim como qualquer outro mercado, as moedas concorrem entre si. Vence aquela que é mais estável, líquida, segura e afiançável. Enquanto a Europa derretia na segunda guerra, o Dólar despontou como a reserva de segurança do mundo.

Mais de 80% das transações do mundo ocorrem em Dólar. 60% das moedas reservadas nos bancos centrais de todo o mundo são dólares.

E quando você precisa negociar algo tão volumoso e volátil quanto milhões de barris de petróleo/dia, acredite, você precisa de uma moeda forte, líquida e abundante.

Bom, é isso: os barris de petróleo são negociados em dólares, porque, provavelmente, essa é a única moeda que consegue suportar o caráter global das negociações; principalmente as diversas trocas cambiais, que, através do dólar, se resumem a apenas duas: da moeda local para o dólar e vice versa.

Quem manda no petróleo?
O mundo consome cerca de 99 milhões de barris de petróleo por dia. Os três maiores produtores são, em ordem: Estados Unidos, Arábia Saudita e Rússia.


O problema é que os americanos consomem quase tudo que produzem, ou seja, estão fora do jogo.
Aqui começa o problema: os americanos consomem quase toda a energia que produzem.

Na verdade, eles conseguem suprir apenas 86% da demanda energética do país. Ou seja, ainda que sejam os maiores produtores do mundo, ao final do mês, precisam importar petróleo.

O que os americanos exportam não é o petróleo em si, mas produtos originados do óleo: polipropileno, vaselina etc.

Por que eu disse que é aqui que começa o problema? Porque isso tira os americanos dos grandes players exportadores do mundo e adiciona volatilidade ao preço do combustível.

Imagine que 50% da produção global exportável está na mão de países de terceiro mundo, instáveis democraticamente e inflacionados.

À menor menção de queda nos preços, a OPEP age, inflacionando os valores artificialmente. Seus países membros precisam disso. O PIB da Arábia Saudita, por exemplo: 80% dele é oriundo da exportação de petróleo.

Imagine que 80% de toda a riqueza do seu país está atrelada ao preço dessa commodity. Como prever investimentos e manter os gastos atrelado à variação diária desse único produto?


Imagine depender da cotação do barril para manter o seu país de pé.
Os grandes produtores de petróleo se esforçam e, desde 2016, removeram 1.8 milhão de barris de petróleo por dia do mercado. Eles criam represas artificiais, onde só extraem mais óleo caso o preço esteja acima de 100 dólares o barril. AINDA QUE HAJA DEMANDA.

Essa medida, contrária às diretrizes da OMC (organização mundial do comércio) irrita principalmente o Presidente Trump, que já manifestou, inúmeras vezes, o seu descontentamento com a situação:


Uma das alegações é que os sauditas mantém 45 milhões de barris de petróleo em seus navios, fora de comercialização.
Chegamos, assim, à primeira resposta do nosso texto: o preço do petróleo é mantido artificialmente alto, ainda que as produções aumentem ano após ano.

Vamos falar de moeda?
Se preço dos combustíveis está intimamente ligado à cotação do Dólar, à exceção dos americanos, o mundo inteiro deve computar a variação da moeda aos preços finais de cada barril.

Os americanos, por estarem em uma economia dolarizada, devem lidar apenas com o aumento ou a queda do preço do óleo. Já o restante do mundo, está fadado ao cálculo:

Valor da alta do barril acrescida à variação cambial do dia.

Agora, chegamos a duas situações distintas:

Você lembra daquela história sobre 50% da reserva mundial estar em países instáveis (tanto politicamente quanto economicamente) e inflacionários? Bom, acrescente também o risco político a essa conta.

O Irã, quinto maior produtor do mundo, se envolve em polêmicas nucleares há pelo menos trinta anos
    """,
    """
   Com o desenvolvimento do ecossistema, inspirado pelo crescimento do Vale do Silício, foi exportado para o mundo inteiro um ideal de empreendedorismo ao estilo Google, só que sem o trabalho duro, o sentido de ownership, as responsabilidades, o coding…

E o que restou? As bicicletas coloridas e as piscinas de bolinha.

Se nos anos oitenta o terno e a gravata representavam o símbolo de status e vitória no mercado de trabalho, hoje em dia a moda é trabalhar vestido de Jedi.

E qual é o problema nisso tudo, afinal de contas?
Bom, é aqui que inicio a minha tese. Pois é, há uma tese. Caso contrário seriam apenas palavras de ódio de algum mal amado, que não curte trabalhar em piscinas de bolinha.

O problema é que, para desenvolver a maior parte das habilidades fundamentais de uma pessoa de negócios bem sucedida, são necessárias três coisas:

A frustração.

A competitividade.

E a agressividade.

Ponto. Essa é a minha tese. Por detrás daquele sorriso de bom moço do Mark Zuckerberg, há um empreendedor agressivo, capaz de ameaçar os concorrentes, copiá-los ou, simplesmente, comprar aquilo que ele ache que pode se tornar um risco ao valor das suas ações.

Atrás daquele rostinho de bom moço do Bill Gates, há um negociador bem sucedido em monopolizar o mercado nos últimos trinta anos. Que teve coragem de expandir para a China quando ninguém quis, tomar e proteger propriedade intelectual da maneira mais laissez faire possível.

Todas as vezes que você rever aquele discurso do Steve Jobs sobre vocação e sobre seguir os seus sonhos, lembre-se que aquelas palavras estão saindo da boca de um dos líderes mais autocrático da cultura americana.

Se, por um lado, agressividade, apreço ao risco e ausência de medo em machucar os outros é uma característica comum entre todos eles, por outro, você não viu nenhum desses caras brincando de pet-day-office enquanto construíam o próprio caminho em direção ao primeiro bilhão.


Esse foi o primeiro escritório do Jeff Bezos, hoje o segundo homem mais rico do mundo.
Essa é a rotina de trabalho do Tim Cook, atual CEO da Apple: “ He’s always been a workaholic; he begins sending emails at 4:30 in the morning. He’s the first in the office and last to leave. He used to hold staff meetings on Sunday night in order to prepare for Monday”.

Bem diferente dos valores pregados pelo ecossistema, não é?

Aliás, bem diferente dos valores pregados pelos próprios caras…bom, sabe por que eles mudaram de opinião? Porque eles já chegaram lá. Porque, agora, eles já estão no topo.

E eu posso apostar um níquel que o motivo deles terem amansado em relação aos seus funcionários, é: porque, assim, a concorrência fica menor.

Quando você sabe que a maioria dos programadores talentosos, designers criativos e redatores de talento estão ocupados assistindo netflix e jogando league of legends, você também sabe que eles serão — para sempre — ótimos funcionários.
    """,
    """
   Como eu sempre gostei de internet, aos 20 anos eu comecei a desenvolver um pequeno programa para finanças. Era uma coisa simples, que te orientava a investir parte do dinheiro que sobrava ao final do mês. O nome do negócio era projeto Midas — algo que você jura que é genial, quando tem essa idade.

Acabei me mudando para Curitiba, atrás de um sujeito que prometeu acelerar o meu projeto em sua agência. Descobri que o cara era louco, que ele fingia psicografar umas cartas da mãe morta de um colega rico, que financiou todo o seu negócio. Fiquei quatro meses lá, até perceber que era uma grande furada e me mandei no meio da madrugada, comprando uma passagem no balcão da companhia aérea.

Antes de partir, um dos gerentes da agência me disse: “Você não aguenta a bomba. Não será nada. Você não será ninguém. O mercado expulsa gente covarde que nem você. Você é fraco e mimado”.

O cara tinha 50 anos, vinte e cinco de mercado, e eu não consegui responder. Voltei devastado, sem dinheiro algum e com o meu namoro quase acabando.

Decidi lançar o projeto sozinho e ele naufragou. Perdi 80 mil reais ao longo do processo. Lancei, dois anos depois, uma versão atualizada do negócio. Ele pareceu estourar, mas, três meses depois, os sócios majoritários me diluíram e me tiraram do projeto, para vendê-lo para uma corretora.

Estava, mais uma vez, de volta à estaca zero.

Abri um comércio no auge da bolha imobiliária. Quando a “marolinha” chegou, eu quebrei.

Montei um software de gestão para uma das maiores empresas de educação a distância do país. Adoraram, mas não renovaram o contrato e botaram estagiários de 18 anos para tocarem a manutenção dele.

Desenhei o negócio de uma empresa de investimentos onde eu só tinha participação nos lucros. Ela foi vendida, para um fundo americano, por alguns milhões de dólares. Não tinha direito a nada.

O meu maior cliente à época, um grupo de construção, me deu um calote de 42 mil reais, depois que o setor estagnou.

A essa altura você deve estar pensando: “Ou esse cara mente ou ele é muito incompetente. Não é possível que alguém se foda tanto!”.

Eu mesmo já me questionei sobre isso. “Será que deveria ter virado médico?”, “A minha namorada vai querer se casar com um fracassado assim?”.

Eu demorei seis anos para perceber que, na verdade, tudo isso era o resultado de uma única coisa: que eu estava tentando — desesperadamente — vencer.

Seis anos depois: “Enfim, o sucesso!”. Calma, não foi bem assim…

Na vida real as coisas não acontecem tão rápido assim…
Enquanto eu quebrava a cara, projeto atrás de projeto, algumas mudanças iam acontecendo dentro de mim — e eu estava tão preocupado em “dar certo”, que eu não percebia.

Ter metido a cara na rua tão cedo me trouxe toneladas de experiências. Enquanto eu, aos 26, já tinha participado de meia dúzia de negócios, dezenas de projetos e ter visto muito dinheiro ir e vir da minha conta corrente, a maioria dos meus amigos ainda estava comemorando o primeiro estágio.

Outro benefício foi: eu estava, finalmente, perdendo o medo do fracasso.

Eu percebia que as pessoas morriam de medo de colocarem os seus negócios na rua e que boa parte desse medo vinha do julgamento das outras pessoas. Ser taxado de fracassado, ser alvo de risadas e julgamentos…

Ter desenhado tantos negócios me trouxe um certo feeling sobre processos. Ter lidado com tanta gente me tornou um bom gestor de pessoas e, depois de tantas páginas — de vendas, de anúncios, lançamentos, novidades — eu melhorei o meu design e desenvolvi habilidades como UX designer.

Enfim, cada fracasso serviu como uma pedra, que agora eram úteis para eu construir o meu castelo.

Parece brega, não é? Pode até ser, mas é a verdade.

Boa parte dos meus amigos gastaram toda a sua juventude tentando evitar erros, fugir dos fracassos e das dores da derrota. O que acabou acontecendo é que eles se tornaram crustáceos: um esqueleto duro, de aparências, escondendo um interior frágil.
    """,
    """
   “Tudo bem, Ícaro. Não seja preconceituoso. Você acabou de conhecer essa senhora. Ainda que ela tenha cagado completamente no título desse artigo e a sua vida amorosa não seja lá um exemplo de resiliência, calma lá. Sejamos coerentes” — conversei com a minha consciência. Coisa de pisciano.

Retorno para o artigo e decido dar um solavanco no meu pré-conceito. A partir de agora eu analisaria tudo a maneira mais imparcial possível. Logo na primeira linha eu encontro uma enquete que, ao ser clicada, revela que 50% das pessoas não acham que vale a pena insistir em um casamento que passa por dificuldades. Que é perda de tempo ou algo do tipo. Enfim, boa parte dos que responderam devem estar juntos para jogar com dois controles no X-box ou para dividir o preço da Netflix. Eis que, antes de terminar o primeiro parágrafo…

Puta merda!

“Ainda não completou um século da entrada do amor no casamento. Foi a partir de 1940 que o matrimônio por amor passou a integrar as relações”. Quantos anos tem essa senhora? Ela tem a moral de, no primeiro fodendo parágrafo, confundir amor com escolha? Fico imaginando o que deve ter dentro de cada um daqueles milhares de livros vendidos.


“O amor tudo sofre, tudo crê, tudo espera, tudo suporta” — Cor. 13:7
O amor sempre esteve presente no Casamento. Com C maiúsculo, inclusive, que é a forma mais fácil de você diferenciar quem realmente entendeu a sua natureza dos meros viajantes e mochileiros sacramentais.

Não é possível que alguém, a essa altura da vida, ainda imagine que, dos anos quarenta para trás, todos se casavam por mera obrigação. Que não só não havia amor prévio, como ele não havia se desenvolvido ao longo de todo o relacionamento. Era uma espécie de cartório doméstico, onde você saía para vender panelas e a sua mulher ficava em casa, limpando o chão e cuidando dos seus trinta e dois fedelhos. Para Regina Navarro, nós somos o resultado do sucesso fantástico do modernismo; uma raça de super-homens, tão bons e tão plenos que fomos capazes de inventar o amor matrimonial.

Seguindo mais um pouco no seu artigo, ela faz algumas comparações bobinhas sobre o que algumas cientistas comportamentais — ou afetivas, sabe-se lá — acreditam que seja ou que não seja o motivo para as pessoas estarem tão infelizes no próprio casamento; mais algumas bobagens, alguns leros e, de repente, mais uma pérola: “O historiador inglês Theodore Zeldin afirma que o medo da solidão assemelha-se a uma bola e uma corrente que, atados a um pé, restringem a ambição, são um obstáculo à vida plena, tal e qual a perseguição, a discriminação e a pobreza”. Eu fico imaginando, para esse sujeito, que caiu de para-quedas nesse artigo, não que ele pudesse atrair outro tipo de coisa melhor, o que é plenitude.


O mano Theo.
Santo Tomás de Aquino havia chegado à conclusão de que o objetivo final da inteligência é amar. Para esse sujeito, endossado pela sexóloga que acabei de conhecer, esse objetivo era outro; algo do tipo fazer o que quiser, em oposição à perseguição, implementar, de vez em quando, o poli-amor, em oposição à discriminação, e ser rico.

Fico imaginando o que esse povo acha de Teresa de Ávila, Padre Damião, Padre Pio ou qualquer irmã que, enclausurada, revela-se uma afronta ao pensamento desse tipo de gente. Eles possuem — exatamente — o contrário do que eles acabaram de elencar: foram pobres, estiveram presos à cordas justíssima durante toda a vida e foram discriminados e perseguidos; entretanto, alguém seria capaz de duvidar da felicidade de cada um deles?

Então, o que falta?
Falta ao homem moderno a compreensão de dever. Retomar a ideia do que é, realmente, o casamento; tanto como Sacramento quanto em relação a sua motivação. Chesterton dizia:
    """,
    """
    Há treze anos eu arruinei os planos do meu pai. Em uma família de médicos, bastaria que eu cumprisse o que já estava pré-acordado, jogando com o regulamento embaixo do braço, para que eu tivesse uma vida tranquila: uma clínica, pacientes e estabilidade, em uma das profissões mais cobiçadas do país.

Eis que eu disse: “Pai, eu não farei medicina”. Até hoje eu não sei se essa frase tinha cinco palavras ou cinco tijolos. Dava para ver nos olhos dele o choque da decisão.

Três ou quatro anos depois o meu pai ainda insistia que eu mudasse de ideia. Para ele, era coisa de menino novo. Na sua cabeça, quando eu percebesse o quão dura era a vida real, eu optaria pelo caminho das pedras.

E, eu posso te dizer uma coisa? Como a vida bate duro!

Eu sentia que precisava fazer alguma faculdade e, como não sabia direito o que escolher, fiz como a maioria dos brasileiros: optei pelo direito. Já na primeira semana eu sabia que aquilo não seria para mim.

Ali eu percebi que o meu futuro não seria em meio as pilhas de papel, filas em fóruns e roupas sociais em um calor de 34 graus.

Lá estava eu, aos vinte anos, tentando descobrir o que eu faria da minha vida.

A vida real: seis anos de quebras e fracassos

Eu fiz de tudo um pouco e não tinha a menor ideia de que, dos 20 aos 26 anos, eu só tomaria no cu. Tudo, literalmente, que eu coloquei a mão deu errado.

Como eu sempre gostei de internet, aos 20 anos eu comecei a desenvolver um pequeno programa para finanças. Era uma coisa simples, que te orientava a investir parte do dinheiro que sobrava ao final do mês. O nome do negócio era projeto Midas — algo que você jura que é genial, quando tem essa idade.

Acabei me mudando para Curitiba, atrás de um sujeito que prometeu acelerar o meu projeto em sua agência. Descobri que o cara era louco, que ele fingia psicografar umas cartas da mãe morta de um colega rico, que financiou todo o seu negócio. Fiquei quatro meses lá, até perceber que era uma grande furada e me mandei no meio da madrugada, comprando uma passagem no balcão da companhia aérea.

Antes de partir, um dos gerentes da agência me disse: “Você não aguenta a bomba. Não será nada. Você não será ninguém. O mercado expulsa gente covarde que nem você. Você é fraco e mimado”.

O cara tinha 50 anos, vinte e cinco de mercado, e eu não consegui responder. Voltei devastado, sem dinheiro algum e com o meu namoro quase acabando.

Decidi lançar o projeto sozinho e ele naufragou. Perdi 80 mil reais ao longo do processo. Lancei, dois anos depois, uma versão atualizada do negócio. Ele pareceu estourar, mas, três meses depois, os sócios majoritários me diluíram e me tiraram do projeto, para vendê-lo para uma corretora.

Estava, mais uma vez, de volta à estaca zero.

Abri um comércio no auge da bolha imobiliária. Quando a “marolinha” chegou, eu quebrei.

Montei um software de gestão para uma das maiores empresas de educação a distância do país. Adoraram, mas não renovaram o contrato e botaram estagiários de 18 anos para tocarem a manutenção dele.

Desenhei o negócio de uma empresa de investimentos onde eu só tinha participação nos lucros. Ela foi vendida, para um fundo americano, por alguns milhões de dólares. Não tinha direito a nada.

O meu maior cliente à época, um grupo de construção, me deu um calote de 42 mil reais, depois que o setor estagnou.

A essa altura você deve estar pensando: “Ou esse cara mente ou ele é muito incompetente. Não é possível que alguém se foda tanto!”.

Eu mesmo já me questionei sobre isso. “Será que deveria ter virado médico?”, “A minha namorada vai querer se casar com um fracassado assim?”.

Eu demorei seis anos para perceber que, na verdade, tudo isso era o resultado de uma única coisa: que eu estava tentando — desesperadamente — vencer.

Seis anos depois: “Enfim, o sucesso!”. Calma, não foi bem assim…

Na vida real as coisas não acontecem tão rápido assim…
Enquanto eu quebrava a cara, projeto atrás de projeto, algumas mudanças iam acontecendo dentro de mim — e eu estava tão preocupado em “dar certo”, que eu não percebia.

Ter metido a cara na rua tão cedo me trouxe toneladas de experiências. Enquanto eu, aos 26, já tinha participado de meia dúzia de negócios, dezenas de projetos e ter visto muito dinheiro ir e vir da minha conta corrente, a maioria dos meus amigos ainda estava comemorando o primeiro estágio.
    """
]

In [ ]:
clonaTomDeVoz(artigos = artigos, groq_api_key = GROQ_API_KEY, model_name = "llama-3.2-90b-vision-preview")

(1/6) Aprendendo o estilo de escrita...
(2/6) Comentando exemplos de utilização do estilo...
(3/6) Gerando o título para o último artigo
(4/6) Criando o artigo utilizando o primeiro estilo de comunicação
(5/6) Refinando o prompt


APIStatusError: Error code: 413 - {'error': {'message': 'Request too large for model `llama-3.2-90b-vision-preview` in organization `org_01j0ty2zweeagtxapyddrz3pwf` on tokens per minute (TPM): Limit 7000, Requested 7392, please reduce your message size and try again. Visit https://console.groq.com/docs/rate-limits for more information.', 'type': 'tokens', 'code': 'rate_limit_exceeded'}}

## Repositório de Prompts (BÔNUS!)
Como bônus também, vou deixar aqui vários lugares onde vocês podem pesquisar exemplos de prompts para utilizar (e estudar!)

- https://github.com/0xeb/TheBigPromptLibrary/
- https://www.reddit.com/r/PromptEngineering/
- https://www.reddit.com/r/LocalLLaMA/
- https://github.com/jujumilk3/leaked-system-prompts
- https://github.com/CyberAlbSecOP/Awesome_GPT_Super_Prompting
- Artigo com os prompts: https://arxiv.org/pdf/2312.16171
- Prompting Caching: https://platform.openai.com/docs/guides/prompt-caching